In [11]:
import pyautogui
from PIL import Image, ImageTk
import tkinter as tk # screen replay
import cv2
import numpy as np

In [4]:
def matchImage(pattern_url, screenshot_url, threshold, lower_range, upper_range, rect_color):
    # Target is in gray already preprocessed beforehand
    pattern_cv2 = cv2.imread(pattern_url)
    tar_gray = cv2.cvtColor(pattern_cv2, cv2.COLOR_BGR2GRAY)


    # Convert to HSV Color Space:
    screenshot_cv2 = cv2.imread(screenshot_url)
    hsv_image = cv2.cvtColor(screenshot_cv2, cv2.COLOR_BGR2HSV)

    # Create and apply the mask:
    mask = cv2.inRange(hsv_image, lower_range, upper_range)
    areas = cv2.bitwise_and(screenshot_cv2, screenshot_cv2, mask=mask)
    
    # Grayscale to remove noise:
    gray_image = cv2.cvtColor(areas, cv2.COLOR_BGR2GRAY)
    
    # Template Matching:
    result = cv2.matchTemplate(gray_image, tar_gray, cv2.TM_CCOEFF_NORMED)

    is_found = False
    loc = np.where(result >= threshold)
    
    # Now Aiming is complete fire down
    if(len(loc[0]) > 0):
        is_found = True
        pt = (loc[1][0], loc[0][0])  # Take the first match
        cv2.rectangle(screenshot_cv2, pt, (pt[0] + pattern_cv2.shape[1], pt[1] + pattern_cv2.shape[0]), rect_color, 2)
    
    return screenshot_cv2, is_found

In [7]:
def checkRadar(pattern_url, screenshot_url, threshold, lower_range, upper_range, rect_color):
    # Target is in gray already preprocessed beforehand
    pattern_cv2 = cv2.imread(pattern_url)
    tar_gray = cv2.cvtColor(pattern_cv2, cv2.COLOR_BGR2GRAY)


    # Convert to HSV Color Space:
    screenshot_cv2 = cv2.imread(screenshot_url)
    height, width, _ = screenshot_cv2.shape
    
    # Calculate the cropping dimensions
    crop_top_percentage = 0.4
    crop_left_percentage = 0.25
    crop_top = int(height * crop_top_percentage)
    crop_left = int(width * crop_left_percentage)

    
    # Define the cropping percentages
    screenshot_cv2 = screenshot_cv2[:crop_top, :crop_left]    
    hsv_image = cv2.cvtColor(screenshot_cv2, cv2.COLOR_BGR2HSV)


    # Define a Color Range:
    mask = cv2.inRange(hsv_image, lower_range, upper_range)
    
    # Apply the Mask:
    areas = cv2.bitwise_and(screenshot_cv2, screenshot_cv2, mask=mask)
    
    # Grayscale to remove noise:
    gray_image = cv2.cvtColor(areas, cv2.COLOR_BGR2GRAY)
    
    # Template Matching:
    result = cv2.matchTemplate(gray_image, tar_gray, cv2.TM_CCOEFF_NORMED)

    is_found = False
    loc = np.where(result >= threshold)
    
    # Now Aiming is complete fire down
    if(len(loc[0]) > 0):
        is_found = True
        pt = (loc[1][0], loc[0][0])  # Take the first match
        cv2.rectangle(screenshot_cv2, pt, (pt[0] + pattern_cv2.shape[1], pt[1] + pattern_cv2.shape[0]), rect_color, 2)
    
    return screenshot_cv2, is_found


In [8]:
# AIM DETECTOR
result_images = []
for i in range(1, 7):
    img, is_present = matchImage("./images/aim_complete.png", "./Aim_train/"+str(i)+".png", 0.25, np.array([50, 218, 183]),  np.array([175, 255, 255]), (0, 0, 255));
    if is_present:
        result_images.append(img)

for idx, result_image in enumerate(result_images, start=1):
    cv2.imshow(f"Matching Result {idx}", result_image)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [9]:
# ENEMY 
result_images = []
for i in range(1, 7):
    img, is_present = matchImage("./images/probable_enemy.png", "./Aim_train/"+str(i)+".png", 0.85, np.array([0, 187, 230]),  np.array([0, 255, 255]), (255, 0, 0))
    if is_present:
        result_images.append(img)

for idx, result_image in enumerate(result_images, start=1):
    cv2.imshow(f"Matching Result {idx}", result_image)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
# ENEMY ON RADAR
result_images = []
for i in range(1, 7):
    img, is_present = checkRadar("./images/radar_tracker.png", "./Aim_train/"+str(i)+".png", 0.17, np.array([0, 187, 230]),  np.array([0, 255, 255]), (0, 255, 0))
    if is_present:
        result_images.append(img)

for idx, result_image in enumerate(result_images, start=1):
    cv2.imshow(f"Matching Result {idx}", result_image)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [10]:
# Collision detection
result_images = []
for i in range(1, 5):
    img, is_present = matchImage("./images/collide_tracker.png", "./Collide_train/"+str(i)+".png", 0.50, np.array([0, 0, 145]),  np.array([161, 43, 255]), (0, 255, 0))
    if is_present:
        result_images.append(img)

for idx, result_image in enumerate(result_images, start=1):
    cv2.imshow(f"Matching Result {idx}", result_image)

cv2.waitKey(0)
cv2.destroyAllWindows()